In [7]:
# import dependencies 
import requests 
import pandas as pd
import numpy as np
import time 
from datetime import datetime
from sqlalchemy import create_engine
import psycopg2
import sqlite3
import config

In [3]:
conn = psycopg2.connect(database="postgres", user=config.db_user, password=config.db_password, sslmode="disable")
cur = conn.cursor();

In [4]:
cur.execute('''SELECT * FROM information_schema.tables where table_schema = 'public' ''')

In [5]:
pd.DataFrame((cur.fetchall()))

,0,1,2,3,4,5,6,7,8,9,10,11
0,postgres,public,notes,BASE TABLE,None,None,None,None,None,YES,NO,None


In [6]:
conn.close()

In [8]:
sqlite_conn = sqlite3.connect("../sqlite/database.sqlite")

In [9]:
sqlite_cur = sqlite_conn.cursor()